In [1]:
import cv2
import numpy as np

#1
roi = None
drag_start = None
mouse_status = 0
tracking_start = False
# 4개 변수 초기화, 마우스 이벤트에 사용

# mouse event 함수 정의
def onMouse(event, x, y, flags, param=None):
    global roi
    global drag_start
    global mouse_status
    global tracking_start
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drag_start = (x, y) # 마우스 드래그 포인트
        mouse_status = 1  # 1은 roi를 설정하기 시작했다는 의미
        tracking_start = False # 아직 트랙킹 안 한 상태
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags == cv2.EVENT_FLAG_LBUTTON:
            xmin = min(x, drag_start[0])
            ymin = min(y, drag_start[1])
            xmax = max(x, drag_start[0])
            ymax = max(y, drag_start[1])
            roi = (xmin, ymin, xmax, ymax)
            mouse_status = 2 # dragging
            # 마우스 시작포인트, 끝 포인트 즉 roi 영역 저장, 드래그 하는 중이라는 의미
            
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_status = 3 # complete 마우스에서 손을 떼었다라는 의미

In [34]:
#2
cv2.namedWindow('tracking') # tracking 이라는 윈도우 생성
cv2.setMouseCallback('tracking', onMouse) # onMous에 등록

path = '/Users/werther/image/'
cap = cv2.VideoCapture(path + 'lucas3.mp4') # 새로운 영상 읽기
if (not cap.isOpened()):
    print('Error opening video')
    
height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), 
                 int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
roi_mask = np.zeros((height, width), dtype=np.uint8) 
# 동영상과 같은 사이즈의 roi 마스크 생성. black

params = dict(maxCorners=16, qualityLevel=0.1, minDistance=1, blockSize=5)
# 밑에 goodFeaturesToTrack에 사용
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
# 계속 돌아가지 않게 종료 조건 설정 시 사용, 10번
params2 = dict(winSize=(5,5), maxLevel=3, criteria=term_crit)
# 루카스 카나데 opticalflow 함수에 넣는 파라미터

#3
t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t += 1
    print('t =', t)
    imgC = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    imgC = cv2.GaussianBlur(imgC, (5,5), 0.0)
    # 프레임 가져와서 gray는 흑백으로 변경 후 가우시안 블러 적용
    
    #3-1
    if mouse_status==2: # 마우스 드래그 중
        x1, y1, x2, y2 = roi
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 2)
    
    #3-2
    if mouse_status==3: # 마우스 손 뗀 상태, roi 설정 완료
        print('initialize...')
        mouse_status = 0
        x1, y1, x2, y2 = roi       # 좌표에 따른 roi 설정
        roi_mask[:, :] = 0         # 원래 블랙 
        roi_mask[y1:y2, x1:x2] = 1 # roi 영역만 1로 셋팅
        p1 = cv2.goodFeaturesToTrack(imgC, mask=roi_mask, **params) # 피처를 찾기 위해 좋은 트랙킹을 찾는다, 최종 피처가 남는다
        if len(p1)>=4:             # 피처가 4개 이상이면
            p1 = cv2.cornerSubPix(imgC, p1, (5,5), (-1,-1), term_crit) # 코너를 찾는다
            rect = cv2.minAreaRect(p1) # 코너가 이루는 최소 사각형 구간
            box_pts = cv2.boxPoints(rect).reshape(-1,1,2) # 포인트로 변경 저장
            tracking_start = True # 트랙킹 시작
            
    #3-3
    if tracking_start: # 트랙킹 시작되었다면,
        p2, st, err = cv2.calcOpticalFlowPyrLK(imgP, imgC, p1, None, **params2) #옵티컬플로우 피라미드 LK
        p1r, st, err = cv2.calcOpticalFlowPyrLK(imgC, imgP, p2, None, **params2)
        # imgC는 가우시안 블러한 현재 프레임, imgP는 현재 프레임을 카피한 것. 반복문이므로 다음 루프 때 이전 루프의 imgC와 동일
        # 즉 imgC는 current frame, imgP는 previous frame
        # p2에 이전 프레임 ~ 현재 프레임 optical flow 저장, p1r에 현재 프레임 ~ 이전 프레임 optical flow 저장
        # 이유는 두 개의 차이가 적어야 optical flow에 문제가 없는 것
        d = abs(p1-p1r).reshape(-1,2).max(-1)
        # 그래서 두 차이를 계산
        stat = d < 1.0 # 1.0 is distance threshold, 차이 기준 1
        good_p2 = p2[stat==1].copy() # p2는 optical flow
        good_p1 = p1[stat==1].copy() # p1은 good feature
        print(height, width)
        print(good_p2)
        for x, y in good_p2.reshape(-1, 2): # p2에 저장되어 있는 것들을 원으로 표시
            cv2.circle(frame, (int(x), int(y)), 3, (0,0,255), -1)
        
        if len(good_p2)<4: # 4보다 적으면 넘어감
            continue
        
        H, mask = cv2.findHomography(good_p1, good_p2, cv2.RANSAC, 3.0) # p1, p2로 네모 찾음 (매칭)
        box_pts = cv2.perspectiveTransform(box_pts, H) # 찾고자 하는 오브젝트의 바운딩 박스 생성
        cv2.polylines(frame, [np.int32(box_pts)], True, (255,0,0), 2) # 바운딩 박스 그림
        p1 = good_p2.reshape(-1,1,2)
        
    #3-4
    cv2.imshow('tracking', frame)
    imgP = imgC.copy()
    key = cv2.waitKey(25)
    if key == 27:
        break

if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

t = 1
1920 1080
[]
t = 2
1920 1080
[]
t = 3
1920 1080
[]
t = 4
1920 1080
[]
t = 5
1920 1080
[]
t = 6
1920 1080
[]
t = 7
1920 1080
[]
t = 8
1920 1080
[]
t = 9
1920 1080
[]
t = 10
1920 1080
[]
t = 11
1920 1080
[]
t = 12
1920 1080
[]
t = 13
1920 1080
[]
t = 14
1920 1080
[]
t = 15
1920 1080
[]
t = 16
1920 1080
[]
t = 17
1920 1080
[]
t = 18
1920 1080
[]
t = 19
1920 1080
[]
t = 20
1920 1080
[]
t = 21
1920 1080
[]
t = 22
1920 1080
[]
t = 23
1920 1080
[]
t = 24
1920 1080
[]
t = 25
1920 1080
[]
t = 26
1920 1080
[]
t = 27
1920 1080
[]
t = 28
1920 1080
[]
t = 29
1920 1080
[]
t = 30
1920 1080
[]
t = 31
1920 1080
[]
t = 32
1920 1080
[]
t = 33
1920 1080
[[[748.95233 839.8077 ]]]
t = 34
1920 1080
[]
t = 35
1920 1080
[]
t = 36
1920 1080
[]
t = 37
1920 1080
[]
t = 38
1920 1080
[]
t = 39
1920 1080
[]
t = 40
1920 1080
[]
t = 41
1920 1080
[]
t = 42
1920 1080
[]
t = 43
1920 1080
[]
t = 44
1920 1080
[]
t = 45
1920 1080
[]
t = 46
1920 1080
[]
t = 47
1920 1080
[]
t = 48
1920 1080
[]
t = 49
1920 1080
[]
t = 50

1920 1080
[[[695.3479  930.4843 ]]

 [[694.56256 910.9334 ]]

 [[789.4794  934.9948 ]]

 [[761.6828  913.47253]]]
t = 292
1920 1080
[[[692.63513 938.7579 ]]

 [[691.9161  918.9675 ]]

 [[786.9777  943.21747]]

 [[759.2229  921.7665 ]]]
t = 293
1920 1080
[[[689.2577  946.73083]]

 [[688.5983  927.16437]]

 [[783.8656  951.32587]]

 [[755.9354  929.6775 ]]]
t = 294
1920 1080
[[[685.8591 954.7728]]

 [[685.0812 934.8481]]

 [[780.5266 959.0694]]

 [[752.5767 937.3136]]]
t = 295
1920 1080
[[[682.0399  962.70715]]

 [[681.2933  942.9322 ]]

 [[776.95935 966.76605]]

 [[748.7755  945.0554 ]]]
t = 296
1920 1080
[[[677.5817  970.21   ]]

 [[676.7605  950.1404 ]]

 [[772.7435  974.17474]]

 [[744.4937  952.3032 ]]]
t = 297
1920 1080
[[[673.2069  978.12354]]

 [[672.3485  957.8961 ]]

 [[768.50653 981.8157 ]]

 [[740.2308  960.01746]]]
t = 298
1920 1080
[[[668.5195  986.233  ]]

 [[667.6434  965.9132 ]]

 [[763.97986 989.7933 ]]

 [[735.6179  967.8724 ]]]
t = 299
1920 1080
[[[663.944   993.65826

TypeError: object of type 'NoneType' has no len()